In [ ]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
filePath = '../data/TCGA2021/data_mrna_seq_v2_rsem.txt'
filePath2 = '../data/TCGA2021/data_rppa.txt'

In [ ]:
df = pd.read_csv(filePath, sep = '\t')
df.head()
print(df.shape)

In [ ]:
df2 = pd.read_csv(filePath2, sep = '\t')
df2.head()
print(df2.shape)

In [ ]:
df2.head()

In [ ]:
df.set_index('Entrez_Gene_Id', inplace=True)
df.drop('Hugo_Symbol', axis=1, inplace=True)
df.columns.name = 'Sample ID'
df.head()

In [ ]:
intersection = set(df.columns).intersection(set(df2.columns))
print(set(df.columns).intersection(set(df2.columns)))
print(f'There are {df.shape[1]} samples in the RNA dataset and {df2.shape[1]} samples in the RPPA dataset. The overlap is size {len(intersection)}')

In [ ]:
df = df.transpose()
nRows, nCols = df.shape
print('Rows:', nRows, 'Columns:', nCols)
mem = df.memory_usage(deep=True).sum() / (1024**2)
print(f'Memory MB: {mem:.2f}')
display(df.head())

In [ ]:
df.dtypes

In [ ]:
sampleSize = 25
randomColumnSample = np.random.choice(nCols, sampleSize, replace=False)

subDf = df.iloc[:,randomColumnSample]

subDf.describe()

In [ ]:
subDf.info()

No numbers appear to be missing...

In [ ]:
constantCols = df.columns[df.var(axis=0) < 1e-10]
numConstantCols = len(constantCols)
print(f'There are {numConstantCols} columns (out of {nCols} total, so {round(numConstantCols/nCols*100, 2)}%) that have constant value; they will be dropped.')

In [ ]:
df = df.drop(columns = constantCols)
nCols = df.shape[1]
print(f'There are {nCols} columns left')

In [ ]:
variances = df.var(axis=0)
nTopGenes = 100
mostVariantGenes=variances.nlargest(nTopGenes).index
print(f'The most "informative" genes were {mostVariantGenes[:5]}')

In [ ]:
from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
df_scaled = pd.DataFrame(std_scaler.fit_transform(df), index=df.index, columns=df.columns)

subDf_scaled = df_scaled.loc[:,mostVariantGenes]

subDf_scaled.describe()

In [ ]:
plt.figure(figsize=(nTopGenes, 50))
cmap = 'vlag'
sns.heatmap(subDf_scaled, cmap=cmap)

plt.show()

In [ ]:

cmap = 'vlag'
sns.clustermap(subDf_scaled, cmap=cmap)

plt.show()

In [ ]:
from sklearn.decomposition import PCA

PCA2D = PCA(n_components=2)
X = PCA2D.fit_transform(df_scaled)
X = pd.DataFrame(X, index=df.index, columns = ['PCA1', 'PCA2'])

plt.scatter(X.iloc[:, 0], X.iloc[:, 1])   
plt.xlabel(X.columns[0])
plt.ylabel(X.columns[1])
plt.show()

In [ ]:
PCA2D.explained_variance_ratio_

In [ ]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

In [ ]:
PCA = PCA(n_components=0.95)
X = pd.DataFrame(PCA.fit_transform(df_scaled), index=df_scaled.index)

PCA.n_components_

In [ ]:
sampleHierarchy = linkage(X, method='ward')

plt.figure(figsize=(15, 4))
dendrogram(sampleHierarchy)
plt.show()

In [ ]:
from scipy.spatial.distance import pdist, squareform

from scipy.cluster.hierarchy import leaves_list

D = squareform(pdist(X, metric='euclidean'))  # or another metric
sampleOrder = leaves_list(sampleHierarchy)
D_reordered = D[np.ix_(sampleOrder, sampleOrder)]

plt.figure(figsize=(15,15))

plt.imshow(D_reordered, cmap='viridis')
plt.title('Distance Heatmap (Clustered Order)')
plt.colorbar()

plt.show()

In [ ]:
geneDF = df_scaled.transpose()
print(geneDF.shape)
geneDF.head()

In [ ]:
topGeneDF = geneDF.loc[mostVariantGenes,:]
topGeneDF.shape

In [ ]:
geneHierarchy = linkage(topGeneDF, method='ward')

plt.figure(figsize=(15, 4))
dendrogram(geneHierarchy)
plt.show()

In [ ]:
geneOrder = leaves_list(geneHierarchy)
smallDF = topGeneDF.transpose()
print(smallDF.shape)


In [ ]:
geneOrder[:5]

In [ ]:
max(geneOrder)

In [ ]:
smallDF_reordered = smallDF.iloc[sampleOrder, geneOrder]

plt.figure(figsize=(20,10))

plt.imshow(smallDF_reordered, cmap='viridis')
plt.xlabel('Genes')
plt.ylabel('Samples')
plt.colorbar()

plt.show()
plt.show()

In [ ]:
sns.clustermap(
    smallDF,
    metric='euclidean', method='ward',
    cmap='bwr',
    figsize=(8, 8)
)